<a href="https://colab.research.google.com/github/BrunoGeraldine/B3_Prediction_2023/blob/main/notebook/EDA_Yfinance_MACD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MACD→ Analise das ações utilizando as médias moveis como orientação para compra e venda.

In [6]:
# Instalando Yfinance API
!pip install plotly

  Using cached plotly-5.14.1-py2.py3-none-any.whl (15.3 MB)
  Using cached tenacity-8.2.2-py3-none-any.whl (24 kB)


In [7]:
# Importando as bibliotecas
import warnings
import numpy                as np
import pandas               as pd
import yfinance             as yf
import seaborn              as sns
import plotly.graph_objects as go

In [8]:
# Help function
# Configuração do padrão de visualização dos dados e graficos
# https://seaborn.pydata.org/tutorial.html
sns.set_theme(
    context='talk',
    style='ticks',
    palette='icefire',
    font_scale=.8,
    rc={'figure.figsize': (25, 15)}
)

warnings.filterwarnings("ignore")
     

In [30]:
# Escolhendo a ação
paper = 'PETR4.SA'

ticker = yf.Ticker(paper)

# Intervalo de tempo
df = ticker.history(period='30d')
df = df['Close']

df_ticker_daily = ticker.history(period='1d', interval='5m')

df_ticker_daily = df_ticker_daily['Close']

In [31]:
info = ticker.info
acao = info['symbol']
acao

'PETR4.SA'

In [32]:
df = pd.DataFrame(df)
df_ticker_daily = pd.DataFrame(df_ticker_daily)

In [33]:
df_ticker_daily.reset_index(inplace=True)

# Quando aplicamos o Yfinance, ele acaba trazendo as informações do momento atual
# e neste momento não queremos isso
#df.drop(df.tail(1).index,inplace=True)
df.reset_index(inplace=True)

In [34]:
df

,Date,Close
0,2023-03-29 00:00:00-03:00,21.281473
1,2023-03-30 00:00:00-03:00,21.343805
2,2023-03-31 00:00:00-03:00,20.880777
3,2023-04-03 00:00:00-03:00,21.806833
4,2023-04-04 00:00:00-03:00,21.610937
5,2023-04-05 00:00:00-03:00,21.682173
6,2023-04-06 00:00:00-03:00,21.370518
7,2023-04-10 00:00:00-03:00,21.824642
8,2023-04-11 00:00:00-03:00,22.848646
9,2023-04-12 00:00:00-03:00,23.017830


In [35]:
# Criando um novo Df com os dados historicos
df_ticker = df

In [36]:
# Remove a ultima data do df_daily
df_remove_ticket = df_ticker.loc[(df_ticker['Date'] == pd.to_datetime('today').normalize())]
df_ticker = df_ticker.drop(df_remove_ticket.index)
df_ticker

,Date,Close
0,2023-03-29 00:00:00-03:00,21.281473
1,2023-03-30 00:00:00-03:00,21.343805
2,2023-03-31 00:00:00-03:00,20.880777
3,2023-04-03 00:00:00-03:00,21.806833
4,2023-04-04 00:00:00-03:00,21.610937
5,2023-04-05 00:00:00-03:00,21.682173
6,2023-04-06 00:00:00-03:00,21.370518
7,2023-04-10 00:00:00-03:00,21.824642
8,2023-04-11 00:00:00-03:00,22.848646
9,2023-04-12 00:00:00-03:00,23.017830


In [37]:
## Coletando o ultimo preço negociado até o agora
#df_ticker_daily_last_price = df_ticker_daily.tail(1)
#df_ticker_daily_last_price

In [38]:
## Ajustar o formato da data e nome coluna
#df_ticker_daily_last_price.rename(columns={'Datetime':'Date'}, inplace=True)
#df_ticker_daily_last_price

In [39]:
#df_ticker_daily_last_price_formated = pd.to_datetime(df_ticker_daily_last_price['Date'], format='%Y-%m-%d')

In [40]:
# Append data atual
#df_ticker = df_ticker.append(df_ticker_daily_last_price)
df_ticker

,Date,Close
0,2023-03-29 00:00:00-03:00,21.281473
1,2023-03-30 00:00:00-03:00,21.343805
2,2023-03-31 00:00:00-03:00,20.880777
3,2023-04-03 00:00:00-03:00,21.806833
4,2023-04-04 00:00:00-03:00,21.610937
5,2023-04-05 00:00:00-03:00,21.682173
6,2023-04-06 00:00:00-03:00,21.370518
7,2023-04-10 00:00:00-03:00,21.824642
8,2023-04-11 00:00:00-03:00,22.848646
9,2023-04-12 00:00:00-03:00,23.017830


In [41]:
#Ajustando a data
df_ticker['Date'] = pd.to_datetime(df_ticker['Date'], utc=True).dt.date
df_ticker

,Date,Close
0,2023-03-29,21.281473
1,2023-03-30,21.343805
2,2023-03-31,20.880777
3,2023-04-03,21.806833
4,2023-04-04,21.610937
5,2023-04-05,21.682173
6,2023-04-06,21.370518
7,2023-04-10,21.824642
8,2023-04-11,22.848646
9,2023-04-12,23.017830


In [42]:
# Calculando o MACD
rapidaMME=df_ticker.Close.ewm(span=12).mean()
lentaMME=df_ticker.Close.ewm(span=26).mean()

MACD = rapidaMME - lentaMME

sinal = MACD.ewm(span=9).mean()

df_ticker['MACD'] = MACD
df_ticker['sinal'] = sinal
df_ticker

,Date,Close,MACD,sinal
0,2023-03-29,21.281473,0.000000,0.000000
1,2023-03-30,21.343805,0.001398,0.000777
2,2023-03-31,20.880777,-0.012577,-0.004696
3,2023-04-03,21.806833,0.014810,0.001912
4,2023-04-04,21.610937,0.021585,0.007764
5,2023-04-05,21.682173,0.028298,0.013330
6,2023-04-06,21.370518,0.017997,0.014511
7,2023-04-10,21.824642,0.032399,0.018810
8,2023-04-11,22.848646,0.092683,0.035875
9,2023-04-12,23.017830,0.142814,0.059836


In [43]:
# Codigo para decidiar compra e venda
df_ticker.at[0, 'flag'] = ''
df_ticker['preco_compra']=np.nan
df_ticker['preco_venda']=np.nan


for i in range(1, len(df_ticker.sinal)):
  if df_ticker['MACD'].iloc[i] > df_ticker['sinal'].iloc[i]:
    if df_ticker['flag'].iloc[i-1] == 'C':
      df_ticker['flag'][i] = 'C'
    else:
      df_ticker['flag'][i] = 'C'
      df_ticker['preco_compra'][i] = df_ticker['Close'][i]

  elif df_ticker['MACD'].iloc[i] < df_ticker['sinal'].iloc[i]:
    if df_ticker['flag'].iloc[i-1] == 'V':
      df_ticker['flag'][i] = 'V'
    else:
      df_ticker['flag'][i] = 'V'
      df_ticker['preco_venda'][i] = df_ticker['Close'][i]

df_ticker


,Date,Close,MACD,sinal,flag,preco_compra,preco_venda
0,2023-03-29,21.281473,0.000000,0.000000,,NaN,NaN
1,2023-03-30,21.343805,0.001398,0.000777,C,21.343805,NaN
2,2023-03-31,20.880777,-0.012577,-0.004696,V,NaN,20.880777
3,2023-04-03,21.806833,0.014810,0.001912,C,21.806833,NaN
4,2023-04-04,21.610937,0.021585,0.007764,C,NaN,NaN
5,2023-04-05,21.682173,0.028298,0.013330,C,NaN,NaN
6,2023-04-06,21.370518,0.017997,0.014511,C,NaN,NaN
7,2023-04-10,21.824642,0.032399,0.018810,C,NaN,NaN
8,2023-04-11,22.848646,0.092683,0.035875,C,NaN,NaN
9,2023-04-12,23.017830,0.142814,0.059836,C,NaN,NaN


In [44]:
#Visualização grafica
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

In [45]:
df_plot = df_ticker

fig = go.Figure()

fig.add_trace(go.Scatter(x=df_plot.index,
                         y=df_plot['Close'],
                         name="Preço fechamento",
                         line_color='orange'
                         ))

fig.add_trace(go.Scatter(x=df_plot.index,
                         y=df_plot['preco_compra'],
                         name="Compra",
                         mode='markers',
                         marker=  dict(
                            color='green',
                            size=10,
                            )
                         ))

fig.add_trace(go.Scatter(x=df_plot.index,
                         y=df_plot['preco_venda'],
                         name="Venda",
                         mode='markers',
                         marker=  dict(
                            color='red',
                            size=10,
                            )
                         ))

fig.show()